In [1]:
import aequilibrae
import os
import sys
import numpy as np
import pandas as pd
import openmatrix as omx
from aequilibrae.matrix import AequilibraeMatrix
from aequilibrae.paths import Graph

In [2]:
root = '/mnt/c/Users/jan.zill/code/TransportationNetworks_aequilibrae'

In [3]:
folders = [x for x in os.listdir(root)[1:] if os.path.isdir(os.path.join(root, x))]

In [18]:
def import_matrix(matfile, do_omx=True, do_aem=True):
    f = open(matfile, 'r')
    all_rows = f.read()
    blocks = all_rows.split('Origin')[1:]
    matrix = {}
    for k in range(len(blocks)):
        orig = blocks[k].split('\n')
        dests = orig[1:]
        orig=int(orig[0])

        d = [eval('{'+a.replace(';',',').replace(' ','') +'}') for a in dests]
        destinations = {}
        for i in d:
            destinations = {**destinations, **i}
        matrix[orig] = destinations
    zones = max(matrix.keys())
    mat = np.zeros((zones, zones))
    for i in range(zones):
        for j in range(zones):
            mat[i, j] = matrix.get(i+1,{}).get(j+1,0)

    index = np.arange(zones) + 1
    
    if do_omx:
        myfile = omx.open_file('demand.omx','w')
        myfile['matrix'] = mat
        myfile.create_mapping('taz', index)
        myfile.close()

    if do_aem:
        aem = AequilibraeMatrix()

        kwargs = {'file_name': 'demand.aem',
                  'zones': zones,
                  'matrix_names': ['matrix']}

        aem.create_empty(**kwargs)
        aem.matrix['matrix'][:,:] = mat[:,:]
        aem.index[:] = index[:]
        aem.close()
    return index

In [124]:
pd.read_csv(i, skiprows=7, sep='\t').columns

Index(['~', 'from ', 'to', 'capacity', 'length', 'ftime', 'B', 'power',
       'speed', 'toll', 'type', 'Unnamed: 11'],
      dtype='object')

In [125]:
i

'ChicagoRegional_net.tntp'

In [131]:
def import_networks(netfile, index):
    
    if 'Anaheim' in netfile:
        data_fields = ['tail', 'head', 'free flow time (min)', 'capacity (veh/h)' ]
    elif 'Birmingham' in netfile:
        data_fields = ['from', 'to', 'free flow time', 'capacity' ]
    elif 'ChicagoSketch' in netfile:
        data_fields = ['tail node', 'head node', 'fftt(min)', 'capacity (veh/h)']
    elif 'Philadelphia_net' in netfile or 'ChicagoRegional_net' in netfile:
        data_fields = ['from', 'to', 'ftime', 'capacity']
    elif 'Sydney_net' in netfile:
        data_fields = ['from', 'to', 'fftt (min)', 'total capacity']
    else:
        data_fields = ['init node', 'term node', 'free flow time', 'capacity']
        
    
    net = pd.read_csv(netfile, skiprows=7, sep='\t')
    
    
    trimmed= [s.strip().lower() for s in net.columns]
    net.columns = trimmed
    
    network = net[data_fields]
    network.columns = ['a_node', 'b_node', 'time', 'capacity']
    network = network.assign(direction=1)
    
    g = Graph()
    g.cost = network['time'].values
    g.capacity = network['capacity'].values
    g.free_flow_time = network['time'].values
    g.network = network.to_records(index=False)
    g.network_ok = True
    g.status = 'OK'
    g.prepare_graph(index)
    g.graph
    g.save_to_disk('graph.aem')

In [132]:
for f in folders:
    mod = os.path.join(root, f)
    os.chdir(mod)
    mod_files = os.listdir('.')
    print(f.upper())

    for i in mod_files:
        if 'TRIPS' in i.upper() and i.lower()[-5:]=='.tntp':
            print('    trips')
            index = import_matrix(i)

    for i in mod_files:
        if '_NET.' in i.upper() and i.lower()[-5:]=='.tntp':
            print('    networks')
            import_networks(i, index)

ANAHEIM
    trips
    networks
AUSTIN
    trips
    networks
BARCELONA
    trips
    networks
BERLIN-CENTER
    trips
    networks
BERLIN-FRIEDRICHSHAIN
    trips
    networks
BERLIN-MITTE-CENTER
    trips
    networks
BERLIN-MITTE-PRENZLAUERBERG-FRIEDRICHSHAIN-CENTER
    trips
    networks
BERLIN-PRENZLAUERBERG-CENTER
    trips
    networks
BERLIN-TIERGARTEN
    trips
    networks
BIRMINGHAM-ENGLAND
    trips
    networks
BRAESS-EXAMPLE
    trips
    networks
CHICAGO-SKETCH
    trips
    networks
EASTERN-MASSACHUSETTS
    trips
    networks
GOLDCOAST
    trips
HESSEN-ASYMMETRIC
    trips
    networks
PHILADELPHIA
    trips
    networks
SIOUXFALLS
    trips
    networks
SYDNEY
    trips
    networks
SYMMETRICATESTCASE
TERRASSA-ASYMMETRIC
    trips
    networks
WINNIPEG-ASYMMETRIC
    trips
    networks
WINNIPEG
    trips
    networks
CHICAGO-REGIONAL
    trips
    networks
